# 房价预测

## 获取和读取数据集

比赛数据分为训练数据集和测试数据集。两个数据集都包括每栋房子的特征，如街道类型，建造年份，房顶类型，地下室状况等特征值。这些特征值有连续的数字，离散只有训练数据集包括了每栋房子的价格，也就是标签。

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import sys 
sys.path.append("..")
import d2lzh_torch as d2l
torch.set_default_tensor_type(torch.FloatTensor)

In [2]:
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

In [6]:
train_data.shape,test_data.shape

((1460, 81), (1459, 80))

In [7]:
train_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]]

,Id,MSSubClass,MSZoning,LotFrontage,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,WD,Normal,208500
1,2,20,RL,80.0,WD,Normal,181500
2,3,60,RL,68.0,WD,Normal,223500
3,4,70,RL,60.0,WD,Abnorml,140000


In [8]:
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))

## 预先数据

我们对连续数值的特征做标准化（standardization）：设该特征在整个数据集上的均值μμ，标准差为σσ。

那么，我们可以设定特征的每个值先输入初始值。

μμ再除以σσ对于缺少的特征值，我们将其替换成该特征的均值。

In [9]:
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 标准化后，每个数值特征的均值变为0，所以可以直接用0来替换缺失值
all_features[numeric_features] = all_features[numeric_features].fillna(0)

接下来将离散数值转成指示特征。

举个例子，假设特征MSZoning里面有两个不同的离散值RL和RM，那么此步转换将去掉MSZoning特征，并新加两个特征MSZoning_RL和MSZoning_RM，其值为0或1。

如果一个样本原来在MSZoning里的变量RL，那么有MSZoning_RL = 1且MSZoning_RM = 0。

In [14]:
all_features = pd.get_dummies(all_features,dummy_na=True)
all_features.shape

(2919, 331)

最后，通过values属性得到NumPy格式的数据，并转成Tensor方便后面的训练。

In [15]:
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float)
test_features = torch.tensor(all_features[n_train:].values, dtype=torch.float)
train_labels = torch.tensor(train_data.SalePrice.values, dtype=torch.float).view(-1, 1)

## 训练模型
我们使用一个基本的线性回归模型和平方损失函数来训练模型。

In [16]:
loss = torch.nn.MSELoss()
def get_net(feature_num):
    net = nn.Linear(feature_num, 1)
    for param in net.parameters():
        nn.init.normal_(param, mean=0, std=0.01)
    return net

In [17]:
def log_rmse(net, features, labels):
    with torch.no_grad():
        # 将小于1的值设成1，使得取对数时数值更稳定
        clipped_preds = torch.max(net(features), torch.tensor(1.0))
        rmse = torch.sqrt(loss(clipped_preds.log(), labels.log()))
    return rmse.item()

下面的训练函数跟本章中前几节的不同在于使用了Adam优化算法。

相对之前使用的小批量随机梯度下降，它对学习率相对不那么敏感。

我们将在之后的“优化算法”一章里详细介绍它

In [ ]:
def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    dataset = torch.utils.data.TensorDataset(train_features, train_labels)
    train_iter = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True)
    # 这里使用了Adam优化算法
    optimizer = torch.optim.Adam(params=net.parameters(), lr=learning_rate, weight_decay=weight_decay) 
    net = net.float()
    for epoch in range(num_epochs):
        for X, y in train_iter:
            l = loss(net(X.float()), y.float())
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
        train_ls.append(log_rmse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))
    return train_ls, test_ls

## K折交叉验证

我们在3.11节（模型选择、欠拟合和过拟合）中介绍了[Math Processing Error]K折交叉验证。

它将被用来选择模型设计并调节超参数。下面实现了一个函数，它返回第i折交叉验证时所需要的训练和验证数据